## Imports

In [38]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd

## Data Read-in

In [39]:
df = pd.read_csv('redfin_2023-09-07-08-26-59.csv')
PBC_gf = gpd.read_file('Palm_Beach_County_Boundary.geojson')

## Data Clean

In [40]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [41]:
df = df.dropna(subset=['SOLD DATE'])

In [42]:
# Define list of desired months (excluding current month)
desired_months = ['August']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [43]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    746
Name: count, dtype: int64
-------
$/SQUARE FEET
False    745
True       1
Name: count, dtype: int64
-------
YEAR BUILT
False    746
Name: count, dtype: int64
-------


In [44]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/West-Palm-Beach/200-Arkona-Ct-33401/unit-2101/home/187291881


In [45]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [46]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [47]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
89,PAST SALE,August-16-2023,Condo/Co-op,2811 S Garden Dr #204,Lake Worth,FL,33461.0,45000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/2811-Gard...,Beaches MLS,RX-10902825,N,Y,26.633652,-80.085226
30,PAST SALE,August-9-2023,Condo/Co-op,335 Northampton Q,West Palm Beach,FL,33417.0,67000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/335-...,Beaches MLS,RX-10899911,N,Y,26.717490,-80.133761
740,PAST SALE,August-7-2023,Condo/Co-op,75 Andover C,West Palm Beach,FL,33417.0,71500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/75-A...,Beaches MLS,RX-10828169,N,Y,26.711927,-80.128382
330,PAST SALE,August-3-2023,Condo/Co-op,9 Norwich A #9,West Palm Beach,FL,33417.0,80000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/9-No...,Beaches MLS,RX-10897530,N,Y,26.720040,-80.120173
596,PAST SALE,August-14-2023,Condo/Co-op,259 Sussex M,West Palm Beach,FL,33417.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/259-...,Beaches MLS,RX-10884243,N,Y,26.720161,-80.134195
667,PAST SALE,August-29-2023,Condo/Co-op,228 Northampton L #228,West Palm Beach,FL,33417.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/228-...,Beaches MLS,F10372625,N,Y,26.717364,-80.134188
193,PAST SALE,August-16-2023,Condo/Co-op,3100 Springdale Blvd #211,Palm Springs,FL,33461.0,80000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Springs/3100-Sp...,Beaches MLS,RX-10894111,N,Y,26.646379,-80.101634
304,PAST SALE,August-18-2023,Condo/Co-op,2300 NE 1st Ln #106,Boynton Beach,FL,33435.0,82000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/2300-N...,Beaches MLS,RX-10896757,N,Y,26.550958,-80.065933
466,PAST SALE,August-29-2023,Condo/Co-op,7754 Lakeside Blvd #433,Boca Raton,FL,33434.0,85000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/7754-Lake...,Beaches MLS,RX-10890643,N,Y,26.379798,-80.168055
635,PAST SALE,August-1-2023,Condo/Co-op,2601 NE 3rd Ct #207,Boynton Beach,FL,33435.0,85000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/2601-N...,Beaches MLS,RX-10875283,N,Y,26.552388,-80.062031


In [48]:
print(df_filtered['URL'].iloc[89])

https://www.redfin.com/FL/Lake-Worth/2811-Garden-Dr-S-33461/unit-204/home/42289185


In [49]:
# # Correct the prices, if needed
# df_filtered.at[245,'PRICE']=(210000)
# df_filtered.at[383,'PRICE']=(225000)
# df_filtered.at[673,'PRICE']=(550000)
# df_filtered.at[777,'PRICE']=(275000)

In [50]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [51]:
# # # Corrections, if needed
# df_filtered.at[245,'$/SQUARE FEET']=(210000/480)
# df_filtered.at[383,'$/SQUARE FEET']=(225000/460)
# df_filtered.at[673,'$/SQUARE FEET']=(550000/960)
# df_filtered.at[777,'$/SQUARE FEET']=(275000/697)

In [52]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
89,45000.0,2811 S Garden Dr #204,Lake Worth,50.0
466,85000.0,7754 Lakeside Blvd #433,Boca Raton,81.0
568,90000.0,6875 Willow Wood Dr #2083,Boca Raton,86.0
600,180000.0,10 Southport Ln Unit G,Boynton Beach,98.0
193,80000.0,3100 Springdale Blvd #211,Palm Springs,103.0
444,176000.0,16032 Lomond Hills Trl #142,Delray Beach,105.0
325,115000.0,7352 W Oakridge Cir Unit 35B,Lake Worth,105.0
574,91000.0,2101 NE 1st Ct #101,Boynton Beach,106.0
304,82000.0,2300 NE 1st Ln #106,Boynton Beach,108.0
717,160000.0,19199 Sabal Lake Dr,Boca Raton,110.0


In [53]:
print(df_filtered.URL.iloc[8])

https://www.redfin.com/FL/Boca-Raton/6845-Willow-Wood-Dr-33434/unit-3031/home/42254394


In [54]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [55]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [56]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [57]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [58]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [59]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [60]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"August 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(PBC_gf,tooltip='Palm Beach County',name='Palm Beach County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [61]:
m.save('index.html')

## Summary Info

In [62]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [63]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [64]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [65]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Lake-Worth/2811-Garden-Dr-S-33461/unit-204/home/42289185


In [66]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Bristol Condo, 1100 S Flagler Dr #804 West Palm Beach | Price $16,000,000 | $3,363 psf | Year built: 2019
Least Expensive
Lake Clarke Gardens Condo 25, 2811 S Garden Dr #204 Lake Worth | Price $45,000 | $50 psf | Year built: 1971
Highest Price Per Square Foot
Sterling Village, 170 Horizons E #108 Boynton Beach | Price $275,000 | $3,667 psf | Year built: 1966
Lowest Price Per Square Foot
Lake Clarke Gardens Condo 25, 2811 S Garden Dr #204 Lake Worth | Price $45,000 | $50 psf | Year built: 1971
Newest
La Clara, 200 Arkona Ct #2201 West Palm Beach | Price $5,500,000 | $1,742 psf | Year built: 2023
Oldest
Palm Beach Hotel Condo, 235 Sunrise Ave #3224 Palm Beach | Price $500,000 | $1,285 psf | Year built: 1925


## Time on Market Calculator

In [69]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Lake-Worth/2811-Garden-Dr-S-33461/unit-204/home/42289185


In [ ]:
from datetime import datetime, timedelta

date1 = datetime(2023, 7, 7) ## List (Earlier) date
date2 = datetime(2023, 8, 30) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

## Map URL Snagger

In [ ]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [ ]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

## Get Summary Data

In [ ]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')